In [2]:
import torch
import tensorly as tl
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import QuantLib as ql
import pysabr
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from scipy.optimize import minimize

%matplotlib inline

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [3]:
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

from tensorly.tenalg import inner

In [27]:
tl.set_backend('pytorch')
batch_size = 10000
device = 'cpu'
kf = KFold(n_splits=4, shuffle=True)

sabr_params = ["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho"]

In [10]:
# scaler = MinMaxScaler()

raw_data = pd.read_csv('fdm_hagan_vol_train.csv')
test_raw_data = pd.read_csv('fdm_hagan_vol_test.csv')
raw_data["Error"] = (raw_data["FDM_vol"] - raw_data["Hagan_vol"])
test_raw_data["Error"] = (test_raw_data["FDM_vol"] - test_raw_data["Hagan_vol"])

X = raw_data.drop(["FDM_vol", "Hagan_vol", "Error", "Strike", "Nu"], axis=1).values
y = raw_data["Error"].values
X_test = test_raw_data.drop(["FDM_vol", "Hagan_vol", "Error"], axis=1).values
y_test = test_raw_data["Error"].values

# scaler.fit(X)
# X = scaler.transform(X)
# X_test = scaler.transform(X_test)


# for train_index, test_index in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [28]:
X = torch.FloatTensor(X)
y = torch.FloatTensor(y)
y = torch.reshape(y, shape=(-1,1))

X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
y_test = torch.reshape(y_test, shape=(-1,1))

In [29]:
train_dataset = torch.utils.data.TensorDataset(X, y)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class HaganDiffNNModel(train_loader, test_loader):
    def __init__(self):
        super(HaganDiffNNModel, self).__init__()
        self.model = nn.Sequential(nn.Linear(6, 20, device=device),
                              nn.Linear(20,100),
                              nn.Linear(100,400),
                              nn.Sigmoid(),
                              nn.Linear(400,10),           
                              nn.Tanh(),
                              nn.Linear(10,1),
                              nn.Tanh()
        )
        self.optimizer = optim.Adam(model.parameters(),lr=0.00001)
        self.n_epochs = 2000
        self.batch_size = 64
    
    def hagan(self, X):
        return np.array([ql.sabrVolatility(row[0], 1, *row[1:]) for row in X])
    
    def train(n_epoch):
        self.model.train()
        for i, (features, error) in enumerate(train_loader):
            features, error = features.to(device), error.to(device)
            self.optimizer.zero_grad()
            output = model(features)
            loss = torch.nn.functional.mse_loss(output,error)
            loss.backward()
            self.optimizer.step()
            if i % 1000 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    self.n_epoch, i * len(features), len(train_loader.dataset),
                    100. * i / len(train_loader), loss))
    
    def test():
        self.model.eval()
        test_loss = 0
        correct = 0
        for features, error in test_loader:
            features, error = features.to(device), error.to(device)
            output = self.model(features)
            test_loss = torch.nn.functional.mse_loss(output,error)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(error.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        print('mean: {}'.format(test_loss))
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
           100. * correct / len(test_loader.dataset)))

    # def forward(self, x):
    #     x = F.relu(F.max_pool2d(self.conv1(x), 2))
    #     x = F.relu(F.max_pool2d(self.conv2(x), 2))
    #     x = self.trl(x)
    #     return F.log_softmax(x)


model = HaganDiffNNModel()
model = model.to(device)

In [42]:
model = nn.Sequential(nn.Linear(6, 20, device=device),
                      nn.Linear(20,100),
                      nn.Linear(100,400),
                      nn.Sigmoid(),
                      nn.Linear(400,10),           
                      nn.Tanh(),
                      nn.Linear(10,1),
                      nn.Tanh()
)

optimizer = optim.Adam(model.parameters(),lr=0.001)
print(model.zero_grad)
n_epochs = 2000
# batch_size = 64

model = model.to(device)


<bound method Module.zero_grad of Sequential(
  (0): Linear(in_features=6, out_features=20, bias=True)
  (1): Linear(in_features=20, out_features=100, bias=True)
  (2): Linear(in_features=100, out_features=400, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=400, out_features=10, bias=True)
  (5): Tanh()
  (6): Linear(in_features=10, out_features=1, bias=True)
  (7): Tanh()
)>


In [ ]:
def train():
    model.train()
    for i, (features, error) in enumerate(train_loader):
        features, error = features.to(device), error.to(device)
        optimizer.zero_grad()
        output = model(features)
        loss = torch.nn.functional.mse_loss(output,error)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(features), len(train_loader.dataset),
                100. * i / len(train_loader), loss))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for features, error in test_loader:
        features, error = features.to(device), error.to(device)
        output = model(features)
        test_loss = torch.nn.functional.mse_loss(output,error)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(error.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('mean: {}'.format(test_loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

for epoch in range(1, 20):
    train(epoch)
    test()


In [31]:
def train(epoch):
    model.train()
    for i, (features, error) in enumerate(train_loader):
        features, error = features.to(device), error.to(device)
        optimizer.zero_grad()
        output = model(features)
        loss = torch.nn.functional.mse_loss(output,error)
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(features), len(train_loader.dataset),
                100. * i / len(train_loader), loss))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    out = pd.DataFrame()
    for features, error in test_loader:
        features, error = features.to(device), error.to(device)
        output = model(features)
        out = out.append(pd.DataFrame(output.detach().numpy()))
        test_loss = torch.nn.functional.mse_loss(output,error)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(error.data.view_as(pred)).cpu().sum()

    # test_loss /= len(test_loader.dataset)
    print('mean: {}'.format(test_loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))
    out.to_csv('one.csv')

for epoch in range(1, 200):
    train(epoch)
    
test()
# train(1)
# test()

Train Epoch: 1 [0/177775 (0%)]	Loss: 0.004812
Train Epoch: 2 [0/177775 (0%)]	Loss: 0.003340
Train Epoch: 3 [0/177775 (0%)]	Loss: 0.001653
Train Epoch: 4 [0/177775 (0%)]	Loss: 0.001737
Train Epoch: 5 [0/177775 (0%)]	Loss: 0.001547
Train Epoch: 6 [0/177775 (0%)]	Loss: 0.001585
Train Epoch: 7 [0/177775 (0%)]	Loss: 0.001587
Train Epoch: 8 [0/177775 (0%)]	Loss: 0.001550
Train Epoch: 9 [0/177775 (0%)]	Loss: 0.001580
Train Epoch: 10 [0/177775 (0%)]	Loss: 0.001565
Train Epoch: 11 [0/177775 (0%)]	Loss: 0.001535
Train Epoch: 12 [0/177775 (0%)]	Loss: 0.001570
Train Epoch: 13 [0/177775 (0%)]	Loss: 0.001535
Train Epoch: 14 [0/177775 (0%)]	Loss: 0.001501
Train Epoch: 15 [0/177775 (0%)]	Loss: 0.001483
Train Epoch: 16 [0/177775 (0%)]	Loss: 0.001460
Train Epoch: 17 [0/177775 (0%)]	Loss: 0.001435
Train Epoch: 18 [0/177775 (0%)]	Loss: 0.001336
Train Epoch: 19 [0/177775 (0%)]	Loss: 0.001286
Train Epoch: 20 [0/177775 (0%)]	Loss: 0.001168
Train Epoch: 21 [0/177775 (0%)]	Loss: 0.001050
Train Epoch: 22 [0/177

In [32]:
output = pd.read_csv('one.csv', usecols=['0'])
test_raw_data = pd.concat([test_raw_data, output],axis=1)
test_raw_data

,Strike,Tenor,Alpha,Beta,Nu,Rho,FDM_vol,Hagan_vol,Error,0
0,1.613705,7.897557,0.400207,0.527964,0.447844,0.244886,0.399076,0.465965,0.399076,0.860249
1,1.703121,5.417643,0.843205,0.358930,0.288882,-0.256672,0.637738,0.731363,0.637738,0.858528
2,0.946012,16.667993,0.130500,0.595940,1.262982,0.046909,0.179535,0.440015,0.179535,0.831281
3,0.521632,6.218122,0.319071,0.490090,0.223082,0.133529,0.384396,0.390709,0.384396,0.857818
4,1.268793,16.224674,0.792654,0.152832,1.336846,-0.148154,0.246090,2.501871,0.246090,0.855235
...,...,...,...,...,...,...,...,...,...,...
49995,0.732634,13.239729,0.517281,0.119887,1.105817,-0.155785,0.287666,1.553630,0.287666,0.862545
49996,0.550702,3.073655,0.773149,0.337450,1.819316,0.360897,0.628061,1.777739,0.628061,0.862465
49997,1.882401,20.934094,0.292555,0.642293,0.907877,0.002441,0.384071,0.918212,0.384071,0.862671
49998,1.611582,13.998001,0.107307,0.324410,1.040509,-0.189993,0.113709,0.418639,0.113709,0.861715


,Strike,Tenor,Alpha,Beta,Nu,Rho,FDM_vol,Hagan_vol,Error,0
0,1.613705,7.897557,0.400207,0.527964,0.447844,0.244886,0.399076,0.465965,0.399076,0.860249
1,1.703121,5.417643,0.843205,0.358930,0.288882,-0.256672,0.637738,0.731363,0.637738,0.858528
2,0.946012,16.667993,0.130500,0.595940,1.262982,0.046909,0.179535,0.440015,0.179535,0.831281
3,0.521632,6.218122,0.319071,0.490090,0.223082,0.133529,0.384396,0.390709,0.384396,0.857818
4,1.268793,16.224674,0.792654,0.152832,1.336846,-0.148154,0.246090,2.501871,0.246090,0.855235
...,...,...,...,...,...,...,...,...,...,...
995,1.034019,4.258776,0.376751,0.140014,1.296823,-0.091058,0.299766,0.592725,0.299766,0.849236
996,1.039462,14.059979,0.322589,0.229697,0.124967,0.084005,0.321435,0.335799,0.321435,0.858151
997,0.835860,6.803596,0.446768,0.342041,1.256399,0.090066,0.320330,0.934428,0.320330,0.849959
998,1.320985,10.631079,0.194139,0.709077,0.620432,-0.329691,0.164034,0.223488,0.164034,0.861669
